In [1]:
import numpy as np
import rebound
import matplotlib.pyplot as plt
import matplotlib
import random
import dill
import sys
import pandas as pd
sys.path.append('../generate_training_data')
from training_data_functions import ressummaryfeaturesxgbv6
%matplotlib inline

folderpath = '../'
model = 'ressummaryfeaturesxgbv6_resonant.pkl'

# Load Model

In [2]:
model, features, featurefolder = dill.load(open(folderpath+'/models/'+model, "rb"))

# Make a random simulation as an example

In [3]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03)
sim.add(m=1.e-5, P=1.2, e=0.03)
sim.add(m=1.e-5, P=1.5, e=0.03)
sim.move_to_com()

# Make the summary features that the model needs to predict. Takes a REBOUND simulation, and a tuple of arguments which shouldn't be changed (These are the values that were used during the training)

In [4]:
%%time
args = (10000, 1000) # (Norbits, Nout) Keep this fixed
summaryfeatures = ressummaryfeaturesxgbv6(sim, args)

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/celmech/celmech/andoyer.py:98: UserWarning: k=1 resonance has no separatrix for Phiprime < 1
  warnings.warn("k=1 resonance has no separatrix for Phiprime < 1")


CPU times: user 1.3 s, sys: 8.94 ms, total: 1.3 s
Wall time: 1.3 s


# We're using the convention that if features = None, then model uses all the features passed back by the function. If features is set, then it will take a subset

In [5]:
if features is not None:
    summaryfeatures = summaryfeatures[features]
summaryfeatures = pd.DataFrame([summaryfeatures]) # convert it to the pandas format model expects

In [6]:
probstability = model.predict_proba(summaryfeatures)[:, 1][0]
print(probstability)

0.0009684455
